# 🎬 Studio-Level AI Video Dubber (Google Colab)

**Features:**
- 🎙️ Multi-speaker voice cloning (Chatterbox + Coqui XTTS)
- 🎵 Professional BGM separation (Demucs mdx_extra_q)
- 🌍 13+ languages supported
- 💋 AI Lip-sync (Wav2Lip)
- ⚡ **GPU-accelerated** on Colab

---

## Instructions:
1. **Enable GPU**: Runtime → Change runtime type → GPU (T4)
2. **Run cells** in order (1 → 2 → 3 → 4)
3. **Download** your dubbed video!

In [ ]:
# @title 📦 Step 1: Clone Repository & Install Dependencies
# @markdown This takes ~3-5 minutes. ☕ Grab some coffee!

import os

# Check GPU
!nvidia-smi --query-gpu=name --format=csv,noheader

# Clone the repo
if not os.path.exists('Video_dubber'):
    print("📥 Cloning repository...")
    !git clone https://github.com/jarair9/Video_dubber.git
    %cd Video_dubber
else:
    print("✅ Repository already cloned")
    %cd Video_dubber

# Install system dependencies
print("\n🔧 Installing system tools...")
!apt-get update -qq
!apt-get install -y -qq espeak-ng ffmpeg

# Install PyTorch with CUDA support (Colab-optimized)
print("\n🔥 Installing PyTorch with GPU support...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install project dependencies
print("\n📚 Installing AI models and dependencies...")
!pip install -q -r requirements-colab.txt

# Verify GPU
import torch
print(f"\n{'='*50}")
print(f"✅ Setup Complete!")
print(f"🔥 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
print(f"{'='*50}")

In [ ]:
# @title 🔑 Step 2: Set API Keys (Required)
# @markdown Add your tokens below. Get them from:
# @markdown - HF Token: https://huggingface.co/settings/tokens
# @markdown - Mistral: https://console.mistral.ai/

import os

# Required for speaker diarization
HF_TOKEN = "" # @param {type:"string"}

# Optional: For LLM-powered translation (or leave empty for Google Translate)
MISTRAL_API_KEY = "" # @param {type:"string"}
OPENROUTER_API_KEY = "" # @param {type:"string"}

# Set environment variables
if HF_TOKEN:
    os.environ["HF_TOKEN"] = HF_TOKEN
    print("✅ HF_TOKEN set (Multi-speaker diarization enabled)")
else:
    print("⚠️ HF_TOKEN not set (will use single-speaker mode)")

if MISTRAL_API_KEY:
    os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY
    print("✅ Mistral API key set")

if OPENROUTER_API_KEY:
    os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY
    print("✅ OpenRouter API key set")

if not MISTRAL_API_KEY and not OPENROUTER_API_KEY:
    print("ℹ️ No LLM API keys - will use Google Translate (free)")

In [ ]:
# @title 📤 Step 3: Upload Your Video
# @markdown Upload an MP4/MKV video file

from google.colab import files
import shutil

print("📤 Click 'Choose Files' to upload your video:")
uploaded = files.upload()

# Get the uploaded file path
input_video = list(uploaded.keys())[0]

# Move to working directory
if not input_video.startswith('/content/'):
    shutil.copy(input_video, f'/content/Video_dubber/{input_video}')
    input_video = f'/content/Video_dubber/{input_video}'

print(f"\n✅ Video ready: {input_video}")
print(f"📊 Size: {os.path.getsize(input_video) / (1024*1024):.1f} MB")

In [ ]:
# @title 🚀 Step 4: Launch Gradio Interface
# @markdown This starts the dubbing app with a web UI. Click the Gradio link to access it.

!python app.py

In [ ]:
# @title 📥 Step 5: Download Output (Alternative CLI Mode)
# @markdown Skip if using Gradio UI above. This runs dubbing via command line.

TARGET_LANGUAGE = "hi" # @param ["en", "hi", "ur", "es", "fr", "de", "ja", "zh", "ko"]
ENABLE_LIPSYNC = True # @param {type:"boolean"}
INCLUDE_BGM = True # @param {type:"boolean"}

# Run dubbing via main.py
!python main.py "{input_video}" --lang {TARGET_LANGUAGE}

# Download the result
import glob
from google.colab import files

output_files = glob.glob("output/*.mp4")
if output_files:
    latest_output = max(output_files, key=os.path.getctime)
    print(f"\n📥 Downloading: {latest_output}")
    files.download(latest_output)
else:
    print("❌ No output found. Check for errors above.")